In [3]:
import json
import spacy
from spacy import displacy
import random
from nltk.corpus import stopwords

In [4]:
stop_words_english = set(stopwords.words('english'))


In [5]:
with open("/home/leon/Desktop/market/data/all_data2.json", "r") as f:
    data = json.load(f)

In [6]:
data["reviews"][1]["review_text"]

'obsessed!!!'

In [14]:
def extract_words(text, spacy_object, stop_words):
    keywords = []
    doc_object = spacy_object(text)
    for token in doc_object:
        if token.pos_ == "ADJ":
            if token.dep_ in ["amod", "conj"]:
                keywords += [token]

        if token.dep_ == "nsubj":
            for child in token.head.children:
                if child.pos_ == "ADJ":
                    keywords += [child]

        if token.dep_ in ["dobj", "acomp"]:
            keywords += [token]
            
        if token.dep_ == "conj" and token.head.pos_ == "VERB":
            keywords += [token]

        if token.dep_ == "advmod" and token.pos_ == "ADV" and token.head.pos_ == "ADJ":
            keywords += [token]
        
        if len([c for c in token.children]) > 3 and token.text not in stop_words:
            keywords += [token]
            
    if len(keywords) == 0:
        keywords = list(doc_object)

    if len(doc_object) == 0 :
        keywords = list(spacy_object("and"))
        
    return(list(set(keywords)))

In [8]:
nlp = spacy.load('en_core_web_md')

In [15]:
index = random.randint(1,5000)
ex = data["reviews"][index]["review_text"]
doc = nlp(ex)
ex

'I just it for the first time last night and although I wished for a different outcome, it was not all that pleasant for me The instant it started to foam, my cheeks and forehead started to sting a bit. I figured, "suck it up and let it work a bit...it\'s probably just tingling from the foam." However, after only 3 minutes, I had to wash it off. I washed it off with COLD water because my face felt like it had been sun burned. It took a while to get off because the clay pack is legit. Certain parts of my face did feel smooth, but overall, I can\'t use this for my whole face. I\'ll likely keep using it for just my nose area where the blackheads are the most prominent, but will keep it away from my more sensitive areas.'

In [16]:
extract_words(ex, nlp, stop_words_english)


[tingling,
 keep,
 burned,
 suck,
 it,
 prominent,
 last,
 area,
 while,
 keep,
 it,
 different,
 let,
 smooth,
 figured,
 Certain,
 started,
 whole,
 pleasant,
 most,
 more,
 use,
 legit,
 it,
 it,
 it,
 first,
 took,
 sensitive,
 feel,
 this,
 washed]

In [9]:
displacy.render(doc, style='dep', jupyter=True)

In [17]:
import numpy as np
import pandas as pd

In [18]:
def get_vector(text, spacy_object, stop_words):
    
    words = extract_words(text, spacy_object, stop_words)
    table = np.array([word.vector for word in words])
    review_vector = np.amax(table, 0)
    
    return (review_vector)


In [19]:
vector_data = []
for x in np.arange(len(data["reviews"])):
    if x % 1000 == 0:
        print(x)
    
    try:
        review_vector = get_vector(data["reviews"][x]["review_text"], nlp, stop_words_english)
        vector_data += [review_vector]
    except:
        print(data["reviews"][x]["review_text"])
        break
    
    

0
1000
Love this stuff! It worked amazing and It got her so fast! I think the pricing was great for this quality​ of a product!


In [15]:
any([len(x) != 300 for x in vector_data])

False

In [43]:
dataframe = pd.DataFrame(np.array(vector_data),columns=np.arange(len(vector_data[0])))

In [45]:
dataframe.to_csv("/home/leon/Desktop/market/data/vector_data.csv", header=True, index=True)

In [46]:
dataframe = pd.read_csv("/home/leon/Desktop/market/data/vector_data.csv", header=0, index_col=0)

In [47]:
ratings = [float(data["reviews"][x]["review_rating"])*30 for x in np.arange(len(data["reviews"]))]

In [49]:
ratings = pd.DataFrame(ratings)

(5570, 1)

In [50]:
master_table = pd.concat([ratings, dataframe], axis=1)

In [52]:
master_table.columns = [x for x in np.arange(301)]
master_table.to_csv("/home/leon/Desktop/market/data/vector_data_ratings.csv", header=True, index=True)